In [1]:
import json
import logging
import time
import random
import os
import requests

from fastapi import FastAPI, Request
import server_apis  # Your helper module for API calls

app = FastAPI()

from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.sequential import SequentialChain

import server_apis

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [25]:
%pip install fastapi uvicorn



Note: you may need to restart the kernel to use updated packages.


In [26]:
email = "ziafigover@gmail.com"
        
api_url = f'{server_apis.SERVER_URL}/mobile-api-100-recommendation-input/{email}'  
user_data = server_apis.fetch_data_with_url(api_url)

api_url_context = f'{server_apis.SERVER_URL}/mobile-api-100-recommendation-input-context/{email}'
context_user_data = server_apis.fetch_data_with_url(api_url_context)


API_URL_Titles = f'{server_apis.SERVER_URL}/slack-api-recommendations-titles/{email}'
prior_habits_impact_response = server_apis.fetch_data_with_url(API_URL_Titles)
prior_habits_impact = prior_habits_impact_response if isinstance(prior_habits_impact_response, str) else json.dumps(prior_habits_impact_response)

API_URL_Change_Titles = f'{server_apis.SERVER_URL}/slack-api-recommendations-titles-change/{email}'
excluded_habits_response = server_apis.fetch_data_with_url(API_URL_Change_Titles)
excluded_habits = excluded_habits_response if isinstance(excluded_habits_response, str) else json.dumps(excluded_habits_response)

API_URL_Preferences = f'{server_apis.SERVER_URL}/slack-api-recommendations-titles-preferences/{email}'
preferences_response = server_apis.fetch_data_with_url(API_URL_Preferences)
preferences = preferences_response if isinstance(preferences_response, str) else json.dumps(preferences_response)

API_URL_Women = f'{server_apis.SERVER_URL}/mobile-api-100-recommendation-input-women/{email}'
women_response = server_apis.fetch_data_with_url(API_URL_Women)
women_data = women_response if isinstance(women_response, str) else json.dumps(women_response)




In [27]:
women_data

'{"menstrual_cycle_data": [{"Question": "last period", "Answer": "19-21 days ago"}, {"Question": "body temprature", "Answer": ""}, {"Question": "full menstrual cycle lenght", "Answer": "31-35 days"}]}'

In [28]:
user_data

{'user_data': [{'TID': '500',
   'Question': 'After work, how do you typically spend your leisure time?',
   'Answer': 'Mostly sedentary (TV/phone',
   'Target': 'Very active (high-intensity workouts'},
  {'TID': '504',
   'Question': 'How much alcohol do you consume after work?',
   'Answer': 'None',
   'Target': '6+ drinks every night'},
  {'TID': '495',
   'Question': 'How often do you feel fatigued or drowsy during the workday?',
   'Answer': 'Most days',
   'Target': 'Never'},
  {'TID': '496',
   'Question': 'How often do you experience neck or back pain related to work posture?',
   'Answer': 'Never',
   'Target': 'Never'},
  {'TID': '502',
   'Question': 'How much time do you typically spend on personal hobbies or creative pursuits after work?',
   'Answer': '3–5 hours per week',
   'Target': '6+ hours per week'},
  {'TID': '498',
   'Question': 'How often do you practice relaxation techniques (deep breathing, meditation, etc.) during work hours?',
   'Answer': 'Never',
   'Targ

In [29]:
context_user_data

{'user_data': [{'Question': 'Kids', 'Answer': 'No'},
  {'Question': 'Mobility', 'Answer': 'Prefer not to say'},
  {'Question': 'gender', 'Answer': 'Female'},
  {'Question': 'optimistic about company', 'Answer': 'Extremely optimistic'},
  {'Question': 'work arrangement',
   'Answer': 'Hybrid (Mix of Office and Remote)'},
  {'Question': 'work from office', 'Answer': 'None (entirely from the home)'},
  {'Question': 'Disability', 'Answer': 'Walking or climbing steps'},
  {'Question': 'Activity: Goal (Preferences)', 'Answer': ''}]}

In [30]:
cycle_prompt = PromptTemplate(
   input_variables=["cycle_context"],
   template="""
You are a health assistant predicting a woman's next menstrual period and cycle phase.


Cycle data:
{cycle_context}


Possible Phases:
- Last period date
- Average cycle length
- Next period start date
- Current phase (e.g., follicular, ovulation, luteal, menstruation)


 {{
   "Period_cycle": "Return only the phase name: Menstrual, Follicular, Ovulation, or Luteal ? output prediction.",
   "Explanition": "What is the prdeicted cycle phase and what sysmtoms could expect good or bad for this phase?",
 }}



"""
)


In [31]:
rec_prompt = PromptTemplate(
   input_variables=["cycle_phase", "user_profile"],
   template="""
You are a wellness and work-life assistant for a woman working in tech.
Here is her recent lifestyle and wellness data:
{user_profile}


Based on this, return a JSON array with 3 structured daily suggestions for the {cycle_phase} phase. Each suggestion should include:
- "title": A short title
- "why": Explanation why it's important with more details and emojies
- "how": How to implement it with more details and emojies
- "try": A practical tip or action with more details and emojies


Only return a valid JSON array like this:


[
 {{
   "title": "Breifly explain the recommendation title",
   "why": "Breifly explain the recommendation why it is important and provide data as references",
   "how": "Breifly explain the recommendation how to implement it",
   "try": "Breifly explain the approach time implementing the recommendation"
 }},
 {{
   "title": "Breifly explain the recommendation title",
   "why": "Breifly explain the recommendation why it is important and provide data as references",
   "how": "Breifly explain the recommendation how to implement it",
   "try": "Breifly explain the approach time implementing the recommendation"
 }},
 {{
  "title": "Breifly explain the recommendation title",
   "why": "Breifly explain the recommendation why it is important and provide data as references",
   "how": "Breifly explain the recommendation how to implement it",
   "try": "Breifly explain the approach time implementing the recommendation"
 }}
]
"""
)


In [32]:
'''
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
import json


# Load menstrual data
cycle_context = women_data


# Load user data
user_profile = user_data
print(user_profile)


# Chain 1: Predict cycle phase
llm = ChatOpenAI(temperature=0.5, openai_api_key=server_apis.OPENAI_API_KEY)
cycle_chain = LLMChain(prompt=cycle_prompt, llm=llm)
cycle_result = cycle_chain.run(cycle_context=cycle_context)
print(cycle_result) 


# Extract phase name
import re
phase_match = re.search(r'(menstrual|follicular|ovulation|luteal)', cycle_result.lower())
cycle_phase = phase_match.group(1).capitalize() if phase_match else "Unknown"


# Chain 2: Personalized structured recommendations
rec_chain = LLMChain(prompt=rec_prompt, llm=llm)
# Parse cycle_result into a dictionary
try:
    cycle_result_dict = json.loads(cycle_result)
except json.JSONDecodeError:
    logger.error("Failed to parse cycle_result as JSON.")
    cycle_result_dict = {}

# Use the parsed dictionary to access 'Period_cycle'
recs_raw = rec_chain.run(cycle_phase=cycle_result_dict.get('Period_cycle', 'Unknown'), user_profile=user_data)



# Try to parse recommendations into JSON array
try:
   recs_json = json.loads(recs_raw)
except json.JSONDecodeError:
   logger.warning("Could not parse recommendations as JSON. Returning raw string.")
   recs_json = recs_raw


# Final structured output
output = {
   "predicted_cycle_phase": cycle_result_dict.get('Period_cycle', 'Unknown'),
   "raw_phase_prediction": cycle_result_dict.get('Explanation', 'Unknown'),
   "personalized_recommendations": recs_json
}




# Print structured JSON output
print("\n=== JSON Output ===")
print(json.dumps(output, indent=4, ensure_ascii=False))


#print("=== Prediction & Phase ===")
#print(cycle_result)
#print("\n=== Personalized Recommendations ===")
#print(recs)
'''


'\nfrom langchain.prompts import PromptTemplate\nfrom langchain.chains import LLMChain\nfrom langchain.chat_models import ChatOpenAI\nimport json\n\n\n# Load menstrual data\ncycle_context = women_data\n\n\n# Load user data\nuser_profile = user_data\nprint(user_profile)\n\n\n# Chain 1: Predict cycle phase\nllm = ChatOpenAI(temperature=0.5, openai_api_key=server_apis.OPENAI_API_KEY)\ncycle_chain = LLMChain(prompt=cycle_prompt, llm=llm)\ncycle_result = cycle_chain.run(cycle_context=cycle_context)\nprint(cycle_result) \n\n\n# Extract phase name\nimport re\nphase_match = re.search(r\'(menstrual|follicular|ovulation|luteal)\', cycle_result.lower())\ncycle_phase = phase_match.group(1).capitalize() if phase_match else "Unknown"\n\n\n# Chain 2: Personalized structured recommendations\nrec_chain = LLMChain(prompt=rec_prompt, llm=llm)\n# Parse cycle_result into a dictionary\ntry:\n    cycle_result_dict = json.loads(cycle_result)\nexcept json.JSONDecodeError:\n    logger.error("Failed to parse cyc

In [33]:
@app.post("/analyze")
async def analyze(request: Request):
    body = await request.json()
    email = body.get("email")

    if not email:
        return {"error": "Email is required."}

    try:
        # Fetch data
        logger.info(f"Fetching data for {email}")
        user_raw = server_apis.fetch_data_with_url(f"{server_apis.SERVER_URL}/mobile-api-100-recommendation-input/{email}")
        cycle_data = server_apis.fetch_data_with_url(f"{server_apis.SERVER_URL}/mobile-api-100-recommendation-input-women/{email}")

        # Format user profile
        user_entries = user_raw.get("user_data", [])
        user_profile = "\n".join([
            f"{entry['Question']} — Answer: {entry['Answer']} (Target: {entry['Target']})"
            for entry in user_entries
        ])

        # LLM setup
        llm = ChatOpenAI(temperature=0.5, openai_api_key=server_apis.OPENAI_API_KEY)

        # Chain 1: Predict phase
        cycle_chain = LLMChain(prompt=cycle_prompt, llm=llm)
        cycle_result = cycle_chain.run(cycle_context=cycle_data)

        try:
            cycle_result_dict = json.loads(cycle_result)
        except json.JSONDecodeError:
            cycle_result_dict = {}

        phase = cycle_result_dict.get("Period_cycle", "Unknown")

        # Chain 2: Recommendations
        rec_chain = LLMChain(prompt=rec_prompt, llm=llm)
        recs_raw = rec_chain.run(cycle_phase=phase, user_profile=user_profile)

        try:
            recs_json = json.loads(recs_raw)
        except json.JSONDecodeError:
            recs_json = recs_raw

        return {
            "predicted_cycle_phase": phase,
            "raw_phase_prediction": cycle_result_dict.get("Explanation", "No explanation provided"),
            "personalized_recommendations": recs_json
        }

    except Exception as e:
        logger.error(f"Error: {e}")
        return {"error": str(e)}